# Домашнее задание NN 3 | Василенко Егор

## Введение (Intro)

### Задание

Реализовать полный RAG-пайплайн, используя фреймворк LangChain. В качестве источника знаний выберите один из двух вариантов:

1. Книга: Любая книга в электронном формате (PDF, TXT, ePub). Это может быть техническая документация, художественное произведение или научная работа.
2. Кодовая база: Исходный код небольшого или среднего проекта. Идеальный вариант - склонировать публичный Git-репозиторий.

После создания RAG-системы вам нужно продемонстрировать её работу на трёх тщательно подобранных ("cherry-picked") примерах запросов, которые показывают сильные и, возможно, слабые стороны вашего решения.

### Импорт библиотек

In [1]:
# Тут много пометок для себя — надеюсь, не будут лишними
# Работа с файловой системой и утилиты
from pathlib import Path
import os
import textwrap # Форматирование текста
from tqdm import tqdm

# Загрузка и обработка документов (LangChain)
from langchain_community.document_loaders import PyMuPDFLoader # Загрузка PDF через PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter # Разбиение текста на чанки

# Векторизация и поиск (LangChain + HuggingFace)
from langchain_huggingface import HuggingFaceEmbeddings # Эмбеддинги через HuggingFace
from langchain_community.vectorstores import Chroma # Векторное хранилище Chroma
from langchain_community.retrievers import BM25Retriever # Текстовый поиск BM25
from langchain.retrievers import EnsembleRetriever # Ансамблевый ретривер

# LLM и генерация ответов (Ollama)
from langchain_ollama import OllamaLLM # LLM через Ollama
from langchain.prompts import PromptTemplate # Шаблоны подсказок
from langchain.chains import RetrievalQA # Цепочка вопрос-ответ с ретривером

## RAG-система

### Конфиг

In [2]:
PDF_PATH = "prestuplenie_i_nakazanie.pdf"
PERSIST_DIR = "chroma_db"
COLLECTION = "book_ru"

# Разбиение
CHUNK_SIZE = 800
CHUNK_OVERLAP = 160

# Эмбеддинги (многоязычные, нормализация обязательна)
EMB_MODEL = "intfloat/multilingual-e5-base"

# Модель LLM в Ollama
OLLAMA_MODEL = "mistral:7b-instruct-q4_K_M"

### Загрузка и предварительная чистка текста

In [3]:
# PyMuPDFLoader обычно чище вытаскивает русский текст из PDF
loader = PyMuPDFLoader(PDF_PATH)
docs = loader.load()
print(f"Страниц загружено: {len(docs)}")

# Мини-чистка: склейка переносов, убираем лишние пробелы
for d in docs:
    t = d.page_content
    t = t.replace("-\n", "") # Переносы со знаком "-"
    t = t.replace("\n", " ") # Переводы строк -> пробел
    d.page_content = " ".join(t.split())

Страниц загружено: 316


In [4]:
# Сплиттер на чанки
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
chunks = splitter.split_documents(docs)
print(f"Чанков получено: {len(chunks)}")

Чанков получено: 1818


### Эмбеддинги и персистентная Chroma (с поддержкой повторного запуска)

In [5]:
# Определяем устройство для эмбеддингов
try:
    import torch
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
except Exception:
    DEVICE = "cpu"

embeddings = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

persist_path = Path(PERSIST_DIR)

if persist_path.exists():
    # Повторный запуск — просто открываем уже созданную коллекцию
    vectorstore = Chroma(
        persist_directory=PERSIST_DIR,
        collection_name=COLLECTION,
        embedding_function=embeddings,
    )
    print("Индекс открыт из персистентного хранилища.")
else:
    # Первый запуск — создаём коллекцию из документов и сохраняем
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=PERSIST_DIR,
        collection_name=COLLECTION,
    )
    vectorstore.persist()
    print("Индекс создан и сохранён.")

C:\Users\MSI\AppData\Local\Temp\ipykernel_31448\3698802203.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Индекс открыт из персистентного хранилища.


### Гибридный ретривер (BM25 + векторный MMR)

In [26]:
# BM25 хорошо ловит точные слова/имена, MMR — разнообразие и смысл
bm25 = BM25Retriever.from_documents(chunks)
bm25.k = 8

vec = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "lambda_mult": 0.5} # Компактнее, меньше шума
)

# Смешиваем веса 0.5/0.5 — честный компромисс; поднимал вес BM25 до 0.7, но на опыте это ничего не дало
retriever = EnsembleRetriever(retrievers=[vec, bm25], weights=[0.5, 0.5])

In [27]:
# Утилита для быстрой подстройки k
def set_k(vec_k: int = 6, bm25_k: int = 8):
    vec.search_kwargs["k"] = vec_k
    bm25.k = bm25_k

In [28]:
# LLM через Ollama
llm = OllamaLLM(
    model=OLLAMA_MODEL,
    temperature=0.15,
    top_p=0.9,
    repeat_penalty=1.1,
    num_ctx=4096,
    num_predict=128
)

### Строгий системный промпт и сборка RetrievalQA

In [29]:
SYS_PROMPT_RAG = """
Ты отвечаешь строго ТОЛЬКО по приведённому ниже КОНТЕКСТУ.
Если нужной информации нет — ответь: "В предоставленном контексте информации нет."
Не цитируй длинные куски и не пересказывай контекст.
Пиши на русском. Для факт-вопросов отвечай кратко: одно имя/словосочетание (1–5 слов), без пояснений.

КОНТЕКСТ:
{context}

ВОПРОС:
{question}

Ответ:
"""

In [30]:
prompt = PromptTemplate(
    template=SYS_PROMPT_RAG,
    input_variables=["context", "question"],
)

In [31]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

### Демонстрация: 3 запроса (факт/синтез/сложный)

In [32]:
queries = {
    "Факт": "Как зовут сестру Раскольникова?",
    "Синтез": "Как изменяется отношение Раскольникова к Сонечке Мармеладовой на протяжении романа? Опиши этапы.",
    "Сложный": "Каким образом в книге упоминается персонаж Гарри Поттер?",
}

for tag, q in queries.items():
    print(f"\n===== {tag.upper()} =====")
    out = qa_chain.invoke({"query": q})
    print("Вопрос:", q)
    print("Ответ :", out["result"])
    print("Контекст:")
    for d in out.get("source_documents", []):
        short = textwrap.shorten(d.page_content.replace("\n"," "), width=110, placeholder="…")
        print(f"  – стр.{d.metadata.get('page_number', '?')}: {short}")


===== ФАКТ =====
Вопрос: Как зовут сестру Раскольникова?
Ответ : 
В предоставленном контексте информации нет.
Контекст:
  – стр.?: . Другого платья у него не было, а если б и было, он, быть может, и не надел бы его, — «так, нарочно бы не…
  – стр.?: !» Катерина Ивановна с презрением заметила, что ее происхождение всем известно и что в этом самом похвальном…
  – стр.?: таясь, открываются оба глаза: они обводят его огненным и бесстыдным взглядом, они зовут его, смеются… Что-то…
  – стр.?: . — А молиться вы умеете? — О, как же, умеем! Давно уже; я, как уж большая, то молюсь сама про себя, а Коля с…
  – стр.?: . Они стали говорить о Лизавете. Студент рассказывал о ней с каким-то особенным удовольствием и все смеялся,…
  – стр.?: . Оба с минуту смотрели друг на друга и ждали. Принесли воды. — Это я … — начал было Раскольников. — Выпейте…
  – стр.?: . Это ты покамест, значит, не хочешь теперь и гораздо важнейшими делами занимаешься… — Дуни дома нет, мамаша?…
  – стр.?: . Но его ни с вами, н

#### Анализ
**Факт**
- *Контекст:* есть упоминание «Дуни» — это сокращённая форма имени Авдотья, то есть нужный ответ фактически содержится в тексте. Однако модель не распознала, что «Дуня» — это сестра Раскольникова, и не выдала его.  
- *Вывод:* ошибка вызвана тем, что поиск выдал релевантный чанк, но синтезатор не связал прозвище с именем. Возможное улучшение — добавить словарь синонимов и уменьшить размер чанков, чтобы уменьшить шум в контексте.

**Синтез**
- *Контекст:* присутствуют описания взаимодействий между героями, но они разрозненные и не дают целостной картины развития отношения. Модель, вероятно, не смогла синтезировать из этих фрагментов полноценный ответ.  
- *Вывод:* ошибка на этапе подбора источников — нужные фрагменты о ключевых этапах отношений не попали в top-k. Для исправления можно увеличить `bm25_k` и добавить более крупные чанк-размеры, чтобы охватить длинные сюжетные линии.

**Сложный**
- *Контекст:* содержит фразы с «каким образом», но ни одного упоминания Гарри Поттера. Это корректный отказ, так как в произведении персонаж не встречается.  
- *Вывод:* модель верно определила отсутствие информации. Здесь улучшений не требуется — поведение соответствует ожидаемому.

### Простая авто-оценка фактов

In [33]:
# Нормализация для честной проверки (ё -> е, регистр, лишние пробелы)
def norm(s: str) -> str:
    return " ".join(s.lower().replace("ё", "е").split())

def evaluate_facts(cases, vec_k: int = 6, bm25_k: int = 8, show_progress: bool = True):
    set_k(vec_k=vec_k, bm25_k=bm25_k)
    ok, results = 0, []
    iterator = tqdm(cases, desc="Оценка фактов", unit="вопр.") if show_progress else cases
    total = len(cases)
    for i, (q, expected) in enumerate(iterator, 1):
        out = qa_chain.invoke({"query": q})
        ans = norm(out["result"])
        ctx = norm(" ".join(d.page_content for d in out.get("source_documents", [])))
        hit = any(norm(e) in ans or norm(e) in ctx for e in expected)
        ok += int(hit)
        results.append((q, out["result"][:160].replace("\n"," "), hit))
        if show_progress:
            iterator.set_postfix_str(f"hit={ok}/{i}")
    acc = ok / total if total else 0.0
    return acc, results

In [34]:
# Небольшой список проверок по роману
fact_cases = [
    ("Как зовут сестру Раскольникова?", ["Авдотья", "Дуня"]),
    ("Как зовут мать Раскольникова?", ["Пульхерия Александровна"]),
    ("Как зовут следователя, беседующего с Раскольниковым?", ["Порфирий Петрович"]),
    ("Как зовут Сонечку по отчеству?", ["Софья Семёновна", "Софья Семеновна"]),
    ("Как зовут помещицу-процентщицу?", ["Алёна Ивановна", "Алена Ивановна"]),
    ("Как зовут её сестру?", ["Лизавета Ивановна"]),
    ("Как звали отца Сонечки?", ["Семён Захарович", "Семен Захарович", "Мармеладов"]),
]

In [35]:
# Запуск оценки
acc, res = evaluate_facts(fact_cases, vec_k=6, bm25_k=8, show_progress=True)
print(f"\nТочность по фактам: {acc*100:.1f}%")
for q, ans, hit in res:
    print(f"[{'OK' if hit else 'MISS'}] {q} -> {ans}")

Оценка фактов: 100%|█████████████████████████████████████████████████████████| 7/7 [00:34<00:00,  4.95s/вопр., hit=2/7]


Точность по фактам: 28.6%
[OK] Как зовут сестру Раскольникова? ->  Лизавета
[MISS] Как зовут мать Раскольникова? ->  В предоставленном контексте информации нет.
[OK] Как зовут следователя, беседующего с Раскольниковым? ->  Неизвестно, потому что в предоставленном контексте информации нет.
[MISS] Как зовут Сонечку по отчеству? ->  В предоставленном контексте информации нет.
[MISS] Как зовут помещицу-процентщицу? ->  В предоставленном контексте информации нет.
[MISS] Как зовут её сестру? ->  В предоставленном контексте информации нет.
[MISS] Как звали отца Сонечки? ->  В предоставленном контексте информации нет.


#### Анализ результата

**Общая точность:** 28.6% (2 верных ответа из 7).

**Наблюдения:**
1. Успехи:  
   - Модель верно определила «Лизавета» как сестру Раскольникова.  
   - Модель дала ответ «Неизвестно...» для вопроса о следователе, что является корректным отказом при отсутствии информации в контексте.
   
2. Проблемы:  
    - В большинстве случаев модель отвечает «В предоставленном контексте информации нет», даже если фрагменты с нужным ответом встречаются в других чанках. Это говорит о недостаточном покрытии контекстом.
    - Есть ошибка в первом успешном примере: «Лизавета» — это на самом деле сестра процентщицы, а не Раскольникова, то есть ответ хоть и из текста, но не тот по смыслу. Это пример галлюцинации из релевантного, но неправильного контекста.
   
***Вывод:***  
- Основная причина низкой точности — непопадание нужных фрагментов в top-k и путаница в идентификации персонажей.
- Для улучшения можно:
    - увеличить `bm25_k` и `vec_k` (например, 8–10);
    - поэкспериментировать с размером чанков;
    - добавить словарь синонимов и проверку ответов по сущностям (NER);
    - использовать иную модель.

## Заключение (Outro)

В ходе выполнения работы была реализована система поиска ответов на вопросы по тексту романа с использованием гибридного подхода: векторного поиска (MMR) и лексического поиска (BM25).

> Для ускорения работы и оптимизации ресурсов добавил проверку на повторный запуск с использованием персистентного хранилища Chroma.

***Проведённая оценка показала точность $28.6%$ по набору тестовых вопросов.***

Анализ ошибок выявил следующие основные проблемы:
- попадание в контекст релевантных, но не относящихся к вопросу фрагментов, что приводит к неправильным фактам (пример: путаница Лизаветы и сестры Раскольникова);
- отсутствие нужной информации в выбранных чанках из-за ограниченного значения `k` и размера чанков;
- склонность модели отвечать «В предоставленном контексте информации нет» при частично релевантных данных.

**Направления для улучшения:**
1. Увеличить значения `bm25_k` и `vec_k` (например, до 8–10) для расширения охвата контекста.
2. Подобрать оптимальный размер чанков и степень их перекрытия.
3. Добавить предобработку ответов с помощью NER для фильтрации нерелевантных имён.
4. Реализовать переформулировку вопросов (query rewriting) для повышения качества поиска.